# Milestone 3 — Rule Firing Debugger (Test)

Pick one test sentence and inspect:
- which patterns fire,
- how they bind `e1`/`e2`,
- which ones pass relaxed anchoring,
- how many different labels they suggest for the same `(e1, e2)` pair.

In [1]:
# Imports / path setup
import json
import sys
from pathlib import Path
from collections import Counter

import spacy
from spacy import displacy

# Make milestone_3/src importable
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from utils import preprocess_data
from execution_engine import (
    compile_dependency_matcher,
    sort_matches_by_priority,
    parse_match_indices,
    verify_anchoring_relaxed,
 )

print('Imports OK')

Successfully imported functions from Milestone 2: /Users/egeaydin/Github/TUW2025WS/Token13-tuw-nlp-ie-2025WS/milestone_2/rule_based
Imports OK


In [2]:
# Load spaCy model
nlp = spacy.load('en_core_web_lg')
print('Loaded:', nlp.meta.get('name'))

Loaded: core_web_lg


In [3]:
# Load TEST data and preprocess
test_path = Path('../../data/processed/test/test.json')
with open(test_path, 'r') as f:
    test_data = json.load(f)

print('Test samples:', len(test_data))
test_processed = preprocess_data(test_data, nlp)
print('Processed:', len(test_processed))

Test samples: 2717


Processing:   0%|          | 0/2717 [00:00<?, ?it/s]

Processed: 2717


In [4]:
# Load patterns + compile DependencyMatcher
patterns_path = Path('../data/patterns_augmented.json')
with open(patterns_path, 'r') as f:
    patterns = json.load(f)

print('Patterns loaded:', len(patterns))
dep_matcher, pattern_lookup = compile_dependency_matcher(patterns, nlp)
print('Compiled patterns:', len(pattern_lookup))

Patterns loaded: 1833
Compiling 1833 patterns into DependencyMatcher...
Successfully compiled 1824 patterns
Compiled patterns: 1824


In [ ]:
def show_tokens(doc, e1_span=None, e2_span=None):
    """Print tokens with indices and mark entity spans."""
    e1_rng = set(range(e1_span.start, e1_span.end)) if e1_span is not None else set()
    e2_rng = set(range(e2_span.start, e2_span.end)) if e2_span is not None else set()
    lines = []
    for t in doc:
        marks = []
        if t.i in e1_rng:
            marks.append('E1')
        if t.i in e2_rng:
            marks.append('E2')
        mark = '[' + ','.join(marks) + ']' if marks else ''
        lines.append(f'{t.i:>3}: {t.text:<15} {mark}')
    print('\n'.join(lines))


def analyze_one(sample_idx=0, top_n=60, max_distance=2, render_deps=False):
    """Analyze which patterns fire for one TEST sample and how they bind e1/e2."""
    sample = test_processed[sample_idx]
    doc = sample['doc']
    e1_span = sample['e1_span']
    e2_span = sample['e2_span']

    print('=' * 80)
    print('Sample', sample_idx)
    print('=' * 80)
    print('Text:', sample['text'])
    print('Actual Relation:', sample['relation'])

    print('E1:', e1_span.text, f'(span {e1_span.start}:{e1_span.end}, root={e1_span.root.i}:{e1_span.root.text})')
    print('E2:', e2_span.text, f'(span {e2_span.start}:{e2_span.end}, root={e2_span.root.i}:{e2_span.root.text})')
    print()
    show_tokens(doc, e1_span, e2_span)

    if render_deps:
        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

    matches = dep_matcher(doc)
    print('\nTotal raw matches:', len(matches))
    if not matches:
        print('No matches fired at all.')
        return

    sorted_matches = sort_matches_by_priority(matches, pattern_lookup, nlp)[:top_n]
    print('Inspecting top matches by priority:', len(sorted_matches))

    def inside_span(idx, span):
        return span.start <= idx <= (span.end - 1)

    anchored_rows = []
    rel_counter = Counter()
    type_counter = Counter()

    for _priority, match_id, token_indices in sorted_matches:
        pat = pattern_lookup[match_id]
        ok = verify_anchoring_relaxed(token_indices, pat, e1_span, e2_span, max_distance=max_distance)
        mi = parse_match_indices(token_indices, pat)
        e1_idx = mi.get('e1')
        e2_idx = mi.get('e2')

        row = {
            'ok': bool(ok),
            'pattern_id': pat.get('pattern_id'),
            'pattern_type': pat.get('pattern_type'),
            'relation': pat.get('relation'),
            'precision': float(pat.get('precision', 0.0)),
            'support': int(pat.get('support', 0)),
            'length': int(pat.get('length', 0)),
            'e1_idx': e1_idx,
            'e2_idx': e2_idx,
            'e1_tok': doc[e1_idx].text if e1_idx is not None else None,
            'e2_tok': doc[e2_idx].text if e2_idx is not None else None,
            'e1_inside': inside_span(e1_idx, e1_span) if e1_idx is not None else False,
            'e2_inside': inside_span(e2_idx, e2_span) if e2_idx is not None else False,
        }

        ok_tag = 'OK' if row['ok'] else 'NO'
        print(
            f"[{ok_tag}] {row['pattern_type']:<14} rel={row['relation']:<28} ",
            f"prec={row['precision']:.2f} sup={row['support']:<3} len={row['length']:<2} ",
            f"e1={row['e1_idx']}({row['e1_tok']}) in={row['e1_inside']} | ",
            f"e2={row['e2_idx']}({row['e2_tok']}) in={row['e2_inside']} | ",
            f"id={row['pattern_id']}",
        )

        if row['ok']:
            anchored_rows.append(row)
            rel_counter[row['relation']] += 1
            type_counter[row['pattern_type']] += 1

    print('\nAnchored matches among inspected:', len(anchored_rows))
    if anchored_rows:
        print('Anchored relation distribution (top 10):')
        for k, v in rel_counter.most_common(10):
            print(f'  {k}: {v}')
        print('Anchored type distribution:')
        for k, v in type_counter.most_common():
            print(f'  {k}: {v}')

        # Decision-list pick = first anchored (because sorted by priority)
        print('\nDecision-list prediction (first anchored):', anchored_rows[0]['relation'])
    else:
        print('\nDecision-list prediction: Other (no anchored match)')

    from collections import defaultdict, Counter

In [27]:
# Pick one test sentence to inspect
sample_idx = 0
top_n = 60
max_distance = 0
render_deps = False  # set True for dependency visualization

analyze_one(sample_idx=sample_idx, top_n=top_n, max_distance=max_distance, render_deps=render_deps)

Sample 0
Text: The most common audits were about waste and recycling.
Actual Relation: Message-Topic
E1: audits (span 3:4, root=3:audits)
E2: waste (span 6:7, root=6:waste)

  0: The             
  1: most            
  2: common          
  3: audits          [E1]
  4: were            
  5: about           
  6: waste           [E2]
  7: and             
  8: recycling       
  9: .               

Total raw matches: 9
Inspecting top matches by priority: 9


UnboundLocalError: cannot access local variable 'Counter' where it is not associated with a value

In [23]:
# Pick one test sentence to inspect
sample_idx = 4
top_n = 60
max_distance = 0
render_deps = False  # set True for dependency visualization

analyze_one(sample_idx=sample_idx, top_n=top_n, max_distance=max_distance, render_deps=render_deps)

Sample 4
Text: Avian influenza is an infectious disease of birds caused by type A strains of the influenza virus.
Actual Relation: Cause-Effect
E1: influenza (span 1:2, root=1:influenza)
E2: virus (span 16:17, root=16:virus)

  0: Avian           
  1: influenza       [E1]
  2: is              
  3: an              
  4: infectious      
  5: disease         
  6: of              
  7: birds           
  8: caused          
  9: by              
 10: type            
 11: A               
 12: strains         
 13: of              
 14: the             
 15: influenza       
 16: virus           [E2]
 17: .               

Total raw matches: 228
Inspecting top matches by priority: 60
[NO] DIRECT         rel=Component-Whole(e2,e1)        prec=1.00 sup=1   len=2   e1=12(strains) in=False |  e2=5(disease) in=False |  id=DIRECT_29787
[NO] TRIANGLE       rel=Content-Container(e1,e2)      prec=1.00 sup=2   len=3   e1=1(influenza) in=True |  e2=15(influenza) in=False |  id=TRIANGLE_57192
[NO]

In [16]:
# Pick one test sentence to inspect
sample_idx = 9
top_n = 60
max_distance = 2
render_deps = False  # set True for dependency visualization

analyze_one(sample_idx=sample_idx, top_n=top_n, max_distance=max_distance, render_deps=render_deps)

Sample 9
Text: This thesis defines the clinical characteristics of amyloid disease.
Actual Relation: Message-Topic
E1: thesis (span 1:2, root=1:thesis)
E2: clinical characteristics (span 4:6, root=5:characteristics)

  0: This            
  1: thesis          [E1]
  2: defines         
  3: the             
  4: clinical        [E2]
  5: characteristics [E2]
  6: of              
  7: amyloid         
  8: disease         
  9: .               

Total raw matches: 29
Inspecting top matches by priority: 29
[OK] TRIANGLE       rel=Message-Topic(e1,e2)          prec=1.00 sup=3   len=3   e1=1(thesis) in=True |  e2=5(characteristics) in=True |  id=TRIANGLE_58932
[NO] BRIDGE         rel=Member-Collection(e2,e1)      prec=1.00 sup=1   len=3   e1=5(characteristics) in=False |  e2=8(disease) in=False |  id=BRIDGE_11391
[NO] BRIDGE         rel=Message-Topic(e1,e2)          prec=1.00 sup=1   len=3   e1=5(characteristics) in=False |  e2=8(disease) in=False |  id=BRIDGE_11270
[NO] BRIDGE         re

In [18]:
# Pick one test sentence to inspect
sample_idx = 11
top_n = 60
max_distance = 0
render_deps = False  # set True for dependency visualization

analyze_one(sample_idx=sample_idx, top_n=top_n, max_distance=max_distance, render_deps=render_deps)

Sample 11
Text: On a friend's advice, I purchased a sauerkraut and kimchi maker here and it is just fabulous (and cheap too).
Actual Relation: Other
E1: kimchi (span 11:12, root=11:kimchi)
E2: maker (span 12:13, root=12:maker)

  0: On              
  1: a               
  2: friend          
  3: 's              
  4: advice          
  5: ,               
  6: I               
  7: purchased       
  8: a               
  9: sauerkraut      
 10: and             
 11: kimchi          [E1]
 12: maker           [E2]
 13: here            
 14: and             
 15: it              
 16: is              
 17: just            
 18: fabulous        
 19: (               
 20: and             
 21: cheap           
 22: too             
 23: )               
 24: .               

Total raw matches: 16
Inspecting top matches by priority: 16
[NO] DIRECT         rel=Product-Producer(e2,e1)       prec=0.44 sup=42  len=2   e1=4(advice) in=False |  e2=2(friend) in=False |  id=DIRECT_1706
[NO] LI

In [ ]:
# Pick one test sentence to inspect
sample_idx = 13
top_n = 60
max_distance = 2
render_deps = False  # set True for dependency visualization

analyze_one(sample_idx=sample_idx, top_n=top_n, max_distance=max_distance, render_deps=render_deps)

from collections import defaultdict, Counter

Sample 13
Text: As a landscape company in Atlanta, we know which plants thrive in this planting zone and know the optimum landscaping designs for local yards and business.
Actual Relation: Product-Producer
E1: landscape (span 2:3, root=2:landscape)
E2: company (span 3:4, root=3:company)

  0: As              
  1: a               
  2: landscape       [E1]
  3: company         [E2]
  4: in              
  5: Atlanta         
  6: ,               
  7: we              
  8: know            
  9: which           
 10: plants          
 11: thrive          
 12: in              
 13: this            
 14: planting        
 15: zone            
 16: and             
 17: know            
 18: the             
 19: optimum         
 20: landscaping     
 21: designs         
 22: for             
 23: local           
 24: yards           
 25: and             
 26: business        
 27: .               

Total raw matches: 292
Inspecting top matches by priority: 60
[NO] DIRECT_2HOP    rel=

NameError: name 'anchored_rows' is not defined